<a href="https://colab.research.google.com/github/amitagarwal2208/Suicide-Intent-Classification/blob/main/Hierarchical_LSTM_SuicideIntentIIITD_AmitAgarwal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed, Dropout
from keras import backend as K
from keras import optimizers
from keras.models import Model
import nltk
import re
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score
from nltk import tokenize
import seaborn as sns

In [128]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
url = 'https://raw.githubusercontent.com/AmanuelF/Suicide-Risk-Assessment-using-Reddit/master/anonymized_dataset/500_Reddit_users_posts_labels.csv'
df = pd.read_csv(url)

In [130]:
df = df.dropna()
X=df.drop('Label',axis=1)
Y=df['Label']

In [131]:
messages = X.copy()

In [132]:
messages = []
exclude = ['[',']',',',', ']

for i in range(500):
  l = df['Post'][i].split("'") 
  l2 = []
  for s in l:
    if s not in exclude:
      l2.append(s)

  messages.append(l2)

In [133]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(500):
  l = []
  for text in messages[i]:
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    l.append(review)

  corpus.append(l) 

In [134]:
def get_max_sentences(corpus):

  maxlen = 0 
  for i in range(len(corpus)):
    maxlen = max(maxlen,len(corpus[i]))

  return maxlen

def get_max_words(corpus):
  maxwords = 0 

  for i in range(len(corpus)):
    for j in range(len(corpus[i])):
      l = corpus[i][j].split(" ")
      maxwords = max(maxwords,len(l))

  return maxwords


def get_total_words(corpus):
  total=0
  for i in range(len(corpus)):
    for j in range(len(corpus[i])):
      l = corpus[i][j].split(" ")
      total+=len(l)

  return total

In [135]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec


In [136]:
total_words = get_total_words(corpus)
tokenizer = Tokenizer(nb_words=total_words)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [137]:
max_num_sentences = get_max_sentences(corpus)
max_num_words = get_max_words(corpus)



In [138]:
X1 = np.zeros((len(corpus),max_num_sentences,max_num_words))

In [139]:
def get_embeddings(corpus):

  tokenizer.fit_on_texts(corpus)
  for i in range(len(corpus)):
    sequences = tokenizer.texts_to_sequences(corpus[i])
    sequences = np.array(sequences)
    data = pad_sequences(sequences, maxlen=max_num_words)
    
    for j in range(data.shape[0]):
      X1[i,j] = data[j]


In [140]:
get_embeddings(corpus)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [141]:
X1[19]

array([[   0.,    0.,    0., ...,    0.,    0.,   12.],
       [   0.,    0.,    0., ..., 7560., 4687., 7560.],
       [   0.,    0.,    0., ..., 4165., 7128.,    9.],
       ...,
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [142]:
Y1 = np.zeros((len(corpus),max_num_sentences,5))

for i in range(len(corpus)):
  idx = -1
  if Y[i] == 'Attempt':
    idx = 0
  elif Y[i] == 'Behavior':
    idx = 1
  elif Y[i] == 'Ideation':
    idx = 2
  elif Y[i] == 'Indicator':
    idx = 3
  elif Y[i] == 'Supportive':
    idx = 4

  for j in range(max_num_sentences):
    Y1[i,j,idx] = 1







In [143]:
X_train = X1[:375,:,:]
X_test = X1[375:,:,:]
Y_train = Y1[:375,:,:]
Y_test = Y1[375:,:,:]

In [144]:
sent_input = Input(shape=(max_num_sentences, max_num_words), dtype='float32')
sent_lstm = (LSTM(150, return_sequences=True, kernel_regularizer="l2"))(sent_input)
sent_dense = TimeDistributed(Dense(200, kernel_regularizer="l2"))(sent_lstm)
print(sent_dense.shape)
sentEncoder = Model(sent_input, sent_dense)


user_input = Input(shape=(max_num_sentences,max_num_words), dtype='float32')
print(user_input.shape)



user_lstm = (LSTM(150, return_sequences=True, kernel_regularizer="l2"))(user_input)
user_dense = TimeDistributed(Dense(200, kernel_regularizer="l2"))(user_lstm)
print(user_dense.shape)
preds = Dense(5, activation='softmax')(user_dense)
#print(preds.shape)
model = Model(user_input, preds)





model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

(None, 292, 200)
(None, 292, 804)
(None, 292, 200)


In [145]:
X_train.shape

(375, 292, 804)

In [146]:
Y_train.shape

(375, 292, 5)

In [150]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [152]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test),epochs=20, batch_size=10)

Epoch 1/20
38/38 [==============================] - 7s 174ms/step - loss: 1.5315 - acc: 0.3488 - val_loss: 1.5303 - val_acc: 0.3200
Epoch 2/20
38/38 [==============================] - 6s 170ms/step - loss: 1.5312 - acc: 0.3482 - val_loss: 1.5326 - val_acc: 0.3141
Epoch 3/20
38/38 [==============================] - 7s 173ms/step - loss: 1.5308 - acc: 0.3453 - val_loss: 1.5328 - val_acc: 0.3099
Epoch 4/20
38/38 [==============================] - 7s 174ms/step - loss: 1.5301 - acc: 0.3486 - val_loss: 1.5290 - val_acc: 0.3172
Epoch 5/20
38/38 [==============================] - 7s 180ms/step - loss: 1.5296 - acc: 0.3489 - val_loss: 1.5294 - val_acc: 0.3197
Epoch 6/20
38/38 [==============================] - 7s 180ms/step - loss: 1.5277 - acc: 0.3484 - val_loss: 1.5317 - val_acc: 0.3132
Epoch 7/20
38/38 [==============================] - 7s 183ms/step - loss: 1.5303 - acc: 0.3484 - val_loss: 1.5306 - val_acc: 0.3134
Epoch 8/20
38/38 [==============================] - 7s 181ms/step - loss: 1.